In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import sqlite3
from sqlalchemy.orm import Session
# Study data files
accidents = "new_data.csv"
# Read the mouse data and the study results
accidents_pd = pd.read_csv(accidents)

# Display the data table for preview
accidents_pd

,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,3429348,A-3436071,Bing,NaN,2,2020-12-29 16:15:00,2020-12-29 23:32:44,34.229923,-117.211105,34.231439,...,False,False,False,False,False,False,Day,Day,Day,Day
1,3804311,A-3811172,Bing,NaN,2,2019-10-19 18:46:00,2019-10-19 22:01:27,34.035712,-118.170145,34.035712,...,False,False,False,False,False,False,Night,Night,Day,Day
2,3380099,A-3386170,Bing,NaN,2,2020-11-21 08:43:00,2020-11-21 12:30:14,38.617427,-121.355262,38.617420,...,False,False,False,False,False,False,Day,Day,Day,Day
3,3003577,A-3004433,Bing,NaN,2,2020-11-20 13:18:00,2020-11-20 15:23:39,34.019522,-118.180044,34.014963,...,False,False,False,False,False,False,Day,Day,Day,Day
4,1218502,A-1218605,MapQuest,201.0,3,2019-09-16 18:56:34,2019-09-16 20:04:33,32.253761,-80.846718,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3530355,A-3537153,Bing,NaN,1,2020-06-08 15:30:47,2020-06-08 16:15:47,32.323240,-111.037420,32.323240,...,False,False,False,False,True,False,Day,Day,Day,Day
4996,3115346,A-3117788,Bing,NaN,2,2020-12-09 05:35:00,2020-12-09 06:50:31,30.997019,-87.488640,30.997953,...,False,False,False,False,False,False,Night,Night,Night,Day
4997,964731,A-964825,MapQuest,201.0,2,2020-02-21 17:07:12,2020-02-21 18:40:14,35.232368,-80.833382,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
4998,935077,A-935170,MapQuest,201.0,3,2020-02-05 08:02:14,2020-02-05 08:31:29,33.427868,-112.036263,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day


In [3]:
data = accidents_pd.loc[:, ["State", "Wind_Speed(mph)","Visibility(mi)","Severity"]]
data

,State,Wind_Speed(mph),Visibility(mi),Severity
0,CA,17.0,7.0,2
1,CA,0.0,10.0,2
2,CA,0.0,10.0,2
3,CA,0.0,10.0,2
4,SC,3.0,10.0,3
...,...,...,...,...
4995,AZ,15.0,10.0,1
4996,FL,0.0,7.0,2
4997,NC,10.0,10.0,2
4998,AZ,6.0,10.0,3


In [113]:
A = data.loc[data['State'] == "AL"]
A.dropna()

,State,Wind_Speed(mph),Visibility(mi),Severity
61,AL,20.0,10.0,3
81,AL,6.0,10.0,2
221,AL,13.0,10.0,2
225,AL,0.0,10.0,3
239,AL,7.0,10.0,2
...,...,...,...,...
4412,AL,8.0,9.0,2
4755,AL,10.4,10.0,2
4786,AL,6.0,10.0,2
4866,AL,0.0,10.0,3


In [10]:
count = accidents_pd.groupby(["State"]).count()["Temperature(F)"]
count

State
AL      84
AR      11
AZ     131
CA    1179
CO      68
CT      30
DC       8
DE       9
FL     428
GA     101
IA      20
ID       5
IL      99
IN      24
KS       9
KY      29
LA     108
MA      44
MD      61
ME       4
MI      84
MN     138
MO      71
MS       2
MT       4
NC     238
ND       1
NE      20
NH      11
NJ      79
NM       7
NV      12
NY     202
OH      72
OK      81
OR     172
PA     161
RI       9
SC     303
SD       1
TN     103
TX     346
UT      72
VA     139
VT       3
WA      71
WI      28
WV       4
Name: Temperature(F), dtype: int64

In [15]:
AL = accidents_pd.loc[:,["State","Wind_Speed(mph)","Temperature(F)"]]
AL

,State,Wind_Speed(mph),Temperature(F)
0,CA,17.0,59.0
1,CA,0.0,70.0
2,CA,0.0,46.0
3,CA,0.0,74.0
4,SC,3.0,82.0
...,...,...,...
4995,AZ,15.0,88.0
4996,FL,0.0,36.0
4997,NC,10.0,43.0
4998,AZ,6.0,37.0


In [29]:
# Optional: Get all the data for the duplicate mouse ID. 
Wind_S = AL.loc[AL["State"]== "WD"]
Wind = Wind_S["Wind_Speed(mph)"]
Wind_Speed = AL_1.dropna()

In [110]:
# Optional: Get all the data for the duplicate mouse ID. 
AL1 = AL.loc[AL["State"]== "WV"]
AL_1 = AL1["Temperature(F)"]
AL_1 = AL_1.dropna()

In [111]:
data_1 = AL_1.T.to_dict()
list(data_1.values())

[79.0, 64.0, 57.0, 61.0]

In [112]:
data_2 = Wind_Speed.T.to_dict()
list(data_2.values())

[73.0,
 78.0,
 75.0,
 75.0,
 79.0,
 69.0,
 78.0,
 76.0,
 45.0,
 48.9,
 86.0,
 89.0,
 75.0,
 86.0,
 85.0,
 79.0,
 77.0,
 84.0,
 64.0,
 59.0,
 79.0,
 61.0,
 75.0,
 66.0,
 61.0,
 82.0,
 70.0,
 79.0,
 78.0,
 81.0,
 75.0,
 59.0,
 77.0,
 31.0,
 73.0,
 65.0,
 75.0,
 88.0,
 68.0,
 73.0,
 77.0,
 78.0,
 68.0,
 94.0,
 53.0,
 82.0,
 79.0,
 77.0,
 67.0,
 54.0,
 70.0,
 79.0,
 63.0,
 91.0,
 82.0,
 75.0,
 73.0,
 68.0,
 69.1,
 60.0,
 57.0,
 73.0,
 84.0,
 75.0,
 75.0,
 70.0,
 73.0,
 83.0,
 64.0,
 58.0,
 73.0,
 76.0,
 77.0,
 78.0,
 88.0,
 73.0,
 86.0,
 81.0,
 75.0,
 88.0,
 73.0,
 48.0,
 84.0,
 74.0,
 79.0,
 46.0,
 84.0,
 67.0,
 73.0,
 80.0,
 63.0,
 84.0,
 80.0,
 83.0,
 72.0,
 74.0,
 75.0,
 84.0,
 87.0,
 59.0,
 72.0,
 45.0,
 64.0,
 88.0,
 82.0,
 57.0,
 81.0,
 68.0,
 75.0,
 84.0,
 60.0,
 80.0,
 76.0,
 78.0,
 81.0,
 49.0,
 72.0,
 89.0,
 73.0,
 54.0,
 74.0,
 73.0,
 68.0,
 70.0,
 55.0,
 74.0,
 74.0,
 80.0,
 91.0,
 57.0,
 77.0,
 81.0,
 76.0,
 82.0,
 79.0,
 59.0,
 73.0,
 82.4,
 72.0,
 54.0,
 77.0,
 87.0,
 86.0,